# 🚗 Extractor de Pólizas Vehiculares (OCR)
Extrae datos de certificados de pólizas desde PDFs escaneados.

**Instrucciones:**
1. Ejecutá cada celda en orden (Shift+Enter o botón ▶️)
2. En el **Paso 2** subí tus archivos PDF
3. En el **Paso 3** se procesan automáticamente
4. En el **Paso 4** descargás el Excel con los resultados

## Paso 1: Instalar dependencias
Esto instala todo lo necesario en el servidor de Google (no en tu PC).

In [ ]:
!apt-get update -qq && apt-get install -y -qq tesseract-ocr > /dev/null 2>&1
!pip install -q pymupdf pytesseract pandas openpyxl Pillow
print('✅ Dependencias instaladas correctamente')

## Paso 2: Subir archivos PDF
Ejecutá esta celda y seleccioná uno o varios archivos PDF.

In [ ]:
from google.colab import files
import os

# Crear carpeta para PDFs
os.makedirs('/content/pdfs', exist_ok=True)

print('📂 Seleccioná tus archivos PDF...')
uploaded = files.upload()

# Mover a carpeta
for nombre in uploaded:
    with open(f'/content/pdfs/{nombre}', 'wb') as f:
        f.write(uploaded[nombre])

archivos = [f for f in os.listdir('/content/pdfs') if f.lower().endswith('.pdf')]
print(f'\n✅ {len(archivos)} archivo(s) PDF subido(s): {archivos}')

## Paso 3: Procesar PDFs (OCR + Extracción)
Esto puede tomar ~1-2 minutos por archivo de 20 páginas.

In [ ]:
import fitz  # PyMuPDF
import pytesseract
from PIL import Image
import pandas as pd
import re
import io
import os

# ============================================================
# CONFIGURACION
# ============================================================
DPI = 300
LANG = 'eng'

# ============================================================
# FUNCIONES
# ============================================================

def pdf_a_imagenes(ruta_pdf, dpi=DPI):
    doc = fitz.open(ruta_pdf)
    imagenes = []
    zoom = dpi / 72
    matriz = fitz.Matrix(zoom, zoom)
    for pagina in doc:
        pix = pagina.get_pixmap(matrix=matriz)
        img = Image.open(io.BytesIO(pix.tobytes('png')))
        imagenes.append(img)
    doc.close()
    return imagenes


def extraer_campo(texto, patrones, grupo=1):
    if isinstance(patrones, str):
        patrones = [patrones]
    for patron in patrones:
        match = re.search(patron, texto, re.IGNORECASE | re.DOTALL)
        if match:
            valor = match.group(grupo).strip()
            return ' '.join(valor.split())
    return None


def extraer_montos(texto):
    resultado = {
        'Suma Asegurada (Todo Riesgo excl. robo)': None,
        'Deducible (Todo Riesgo excl. robo)': None,
        'Suma Asegurada (Robo y/o Hurto Total)': None,
        'Deducible (Robo y/o Hurto Total)': None,
    }
    seccion = re.search(
        r'(?:Todo Riesgo excluyendo robo|excluyendo robo)(.*)',
        texto, re.IGNORECASE | re.DOTALL
    )
    if not seccion:
        return resultado
    bloque = seccion.group(1)
    montos_raw = re.findall(r'([\d,]+\.\d{2})', bloque)
    montos = []
    for m in montos_raw:
        try:
            montos.append(float(m.replace(',', '')))
        except ValueError:
            continue
    if len(montos) >= 4:
        resultado['Suma Asegurada (Todo Riesgo excl. robo)'] = montos[0]
        resultado['Deducible (Todo Riesgo excl. robo)'] = montos[1]
        resultado['Suma Asegurada (Robo y/o Hurto Total)'] = montos[2]
        resultado['Deducible (Robo y/o Hurto Total)'] = montos[3]
    elif len(montos) == 2:
        resultado['Suma Asegurada (Todo Riesgo excl. robo)'] = montos[0]
        resultado['Suma Asegurada (Robo y/o Hurto Total)'] = montos[1]
    elif len(montos) == 1:
        resultado['Suma Asegurada (Todo Riesgo excl. robo)'] = montos[0]
    return resultado


def extraer_certificado(texto_pagina):
    if not texto_pagina or 'DATOS GENERALES' not in texto_pagina:
        return None
    d = {}
    d['Ramo'] = extraer_campo(texto_pagina, r'Ramo:\s*(\w+)')
    d['Tipo de Cobertura'] = extraer_campo(texto_pagina, [
        r'Tipo de cobertura:\s*(.+?)(?:\s{2,}|P[oó]liza)',
        r'cobertura:\s*(.+?)(?:\s{2,}|Poliza)',
    ])
    d['No. Poliza'] = extraer_campo(texto_pagina, r'P[oó]liza No\.?:\s*(.+?)(?:\s{2,}|Certificado|\n)')
    d['No. Certificado'] = extraer_campo(texto_pagina, r'Certificado No\.?:?\s*(\d+)')
    d['Inicio Vigencia'] = extraer_campo(texto_pagina, r'Inicio de Vigencia:\s*([\d/]+)')
    d['Fin Vigencia'] = extraer_campo(texto_pagina, r'Fin de Vigencia:\s*([\d/]+)')
    d['Asegurado'] = extraer_campo(texto_pagina, r'Asegurado:\s*(.+?)(?:\s{2,}|Descripci[oó]n|\n)')
    if d['Asegurado'] and 'SOCIEDAD' in d['Asegurado'] and 'ANONIMA' not in d['Asegurado']:
        d['Asegurado'] += ' ANONIMA'
    d['Propietario'] = extraer_campo(texto_pagina, r'Propietario:\s*(.+?)(?:\s{2,}|L[ií]nea|\n)')
    if d['Propietario'] and 'SOCIEDAD' in d['Propietario'] and 'ANONIMA' not in d['Propietario']:
        d['Propietario'] += ' ANONIMA'
    d['NIT'] = extraer_campo(texto_pagina, r'Nit:\s*(\d+)')
    d['Direccion'] = extraer_campo(texto_pagina, r'Direcci[oó]n:\s*(.+?)(?:\n|Color)')
    d['Marca'] = extraer_campo(texto_pagina, r'Marca:\s*([A-Z!]\w*)')
    if d['Marca']:
        correcciones = {'!SUZU': 'ISUZU', '!suzu': 'ISUZU'}
        d['Marca'] = correcciones.get(d['Marca'], d['Marca'])
    d['Linea'] = extraer_campo(texto_pagina, r'L[ií]nea:\s*(.+?)(?:\s{2,}|Pasajeros|\n)')
    d['Modelo (Ano)'] = extraer_campo(texto_pagina, r'Modelo:\s*(\d{4})')
    d['Clase'] = extraer_campo(texto_pagina, r'Clase:\s*(.+?)(?:\s{2,}|\n)')
    d['Pasajeros'] = extraer_campo(texto_pagina, r'Pasajeros:\s*(\d+)')
    d['Placa'] = extraer_campo(texto_pagina, r'Placa:\s*([A-Z0-9][\s\-]*[\d\w]+)')
    if d['Placa']:
        d['Placa'] = d['Placa'].replace(' ', '')
    d['Chasis'] = extraer_campo(texto_pagina, r'Chasis:\s*(\S+)')
    d['Motor'] = extraer_campo(texto_pagina, r'Motor:\s*(.+?)(?:\s{2,}|\n)')
    d['Color'] = extraer_campo(texto_pagina, r'Color:\s*(.+?)(?:\s{2,}|\n)')
    d['Codigo Agente'] = extraer_campo(texto_pagina, r'[Cc][oó]digo del agente:?\s*(\d+)')
    d['Nombre Agente'] = extraer_campo(texto_pagina, [
        r'Nombre del agente:?\s*(.+?)(?:\n|$)',
        r'Nombre d\w+ agente:?\s*(.+?)(?:\n|$)',
    ])
    d['Forma de Pago'] = extraer_campo(texto_pagina, r'Forma de Pago:\s*(\w+)')
    d['Moneda'] = extraer_campo(texto_pagina, r'Moneda:\s*(\w+)')
    d.update(extraer_montos(texto_pagina))
    return d


# ============================================================
# PROCESAR TODOS LOS PDFs
# ============================================================
carpeta = '/content/pdfs'
archivos_pdf = sorted([f for f in os.listdir(carpeta) if f.lower().endswith('.pdf')])
print(f'Procesando {len(archivos_pdf)} archivo(s)...\n')

todos_certificados = []

for archivo in archivos_pdf:
    ruta = os.path.join(carpeta, archivo)
    print(f'📄 {archivo}')
    try:
        imagenes = pdf_a_imagenes(ruta, dpi=DPI)
        print(f'   {len(imagenes)} paginas encontradas')
        for i, img in enumerate(imagenes):
            texto = pytesseract.image_to_string(img, lang=LANG)
            datos = extraer_certificado(texto)
            if datos:
                datos['Archivo Origen'] = archivo
                datos['Pagina'] = i + 1
                todos_certificados.append(datos)
                cert = datos.get('No. Certificado', '?')
                marca = datos.get('Marca', '?')
                placa = datos.get('Placa', '?')
                print(f'   ✅ Pag {i+1}: Cert #{cert} | {marca} | {placa}')
        print()
    except Exception as e:
        print(f'   ❌ Error: {e}\n')

print(f'\n🎯 Total: {len(todos_certificados)} certificados extraidos')

## Paso 4: Exportar y descargar resultados
Genera el Excel y CSV, y los descarga automáticamente a tu PC.

In [ ]:
from google.colab import files as colab_files

if not todos_certificados:
    print('⚠️ No se encontraron certificados para exportar.')
else:
    df = pd.DataFrame(todos_certificados)

    # Ordenar columnas
    columnas_orden = [
        'Archivo Origen', 'Pagina', 'No. Poliza', 'No. Certificado',
        'Ramo', 'Tipo de Cobertura', 'Inicio Vigencia', 'Fin Vigencia',
        'Asegurado', 'Propietario', 'NIT', 'Direccion',
        'Marca', 'Linea', 'Modelo (Ano)', 'Clase', 'Pasajeros',
        'Placa', 'Chasis', 'Motor', 'Color',
        'Suma Asegurada (Todo Riesgo excl. robo)', 'Deducible (Todo Riesgo excl. robo)',
        'Suma Asegurada (Robo y/o Hurto Total)', 'Deducible (Robo y/o Hurto Total)',
        'Codigo Agente', 'Nombre Agente', 'Forma de Pago', 'Moneda'
    ]
    cols = [c for c in columnas_orden if c in df.columns]
    cols += [c for c in df.columns if c not in columnas_orden]
    df = df[cols]

    # Exportar Excel
    xlsx = '/content/polizas_extraidas.xlsx'
    with pd.ExcelWriter(xlsx, engine='openpyxl') as writer:
        df.to_excel(writer, index=False, sheet_name='Certificados')
        ws = writer.sheets['Certificados']
        for i, col in enumerate(df.columns, 1):
            w = max(len(str(col)), df[col].astype(str).str.len().max() if len(df) else 0)
            ws.column_dimensions[ws.cell(1, i).column_letter].width = min(w + 2, 45)

    # Exportar CSV
    csv_path = '/content/polizas_extraidas.csv'
    df.to_csv(csv_path, index=False, encoding='utf-8-sig')

    # Resumen
    print('=' * 50)
    print(f'  Certificados: {len(df)}')
    print(f'  Archivos:     {df["Archivo Origen"].nunique()}')
    if 'Marca' in df.columns:
        print(f'  Marcas:       {chr(44).join(" " + m for m in df["Marca"].dropna().unique())}')
    if 'Suma Asegurada (Todo Riesgo excl. robo)' in df.columns:
        total = df['Suma Asegurada (Todo Riesgo excl. robo)'].sum()
        print(f'  Suma Total:   Q {total:,.2f}')
    print('=' * 50)

    # Mostrar tabla preview
    display(df[['No. Certificado', 'Marca', 'Linea', 'Modelo (Ano)', 'Placa',
                'Suma Asegurada (Todo Riesgo excl. robo)', 'Deducible (Todo Riesgo excl. robo)']])

    # Descargar archivos
    print('\n📥 Descargando archivos...')
    colab_files.download(xlsx)
    colab_files.download(csv_path)

---
## 🔄 ¿Más archivos?
Si necesitás procesar más PDFs, volvé al **Paso 2** y subí los nuevos archivos.
Luego ejecutá **Paso 3** y **Paso 4** de nuevo.